In [1]:
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError
from sklearn.manifold import TSNE

import numpy as np
import altair as alt
import pandas as pd
import os

from module.neo4j.graph_db import GraphDB
driver = GraphDatabase.driver("bolt://localhost:11012", auth=("neo4j", "erclab"))

In [6]:
with driver.session() as session:
    session.run("""
                   CALL gds.graph.create(
                   'recommendation_embeddings',
                   {
                    User: {
                        label: 'User'
                    },
                    Restaurant: {
                        label: 'Restaurant',
                        properties: ['rating']
                    }
                   },{
                    RATE: {
                        type: 'RATE',
                        orientation: 'UNDIRECTED',
                        properties: ['star']
                    }
                    },
                    {nodeProperties:embeddings})
                   YIELD graphName, nodeCount, relationshipCount
                   RETURN graphName, nodeCount, relationshipCount""")

In [4]:
with driver.session() as session:
    session.run("""
        UNWIND range(0,49) as i
        WITH collect('embedding_' + toString(i)) as embeddings
        CALL gds.beta.graphSage.train('train',{
          modelName:'foodModel',
          aggregator:'pool',
          batchSize:512,
          activationFunction:'relu',
          epochs:10,
          sampleSizes:[25,10],
          learningRate:0.0000001,
          embeddingDimension:256,
          featureProperties:embeddings})
        YIELD modelInfo
        RETURN modelInfo""")

In [5]:
with driver.session() as session:
    session.run("""
        UNWIND range(0,49) as i
        WITH collect('embedding_' + toString(i)) as embeddings
        CALL gds.graph.create('test','Test',{INTERACTS:{orientation:'UNDIRECTED'}},
          {nodeProperties:embeddings})
        YIELD graphName, nodeCount, relationshipCount
        RETURN graphName, nodeCount, relationshipCount""")